In [1]:
# PennyLane imports
import pennylane as qml
from pennylane import numpy as pnp

from scipy.optimize import differential_evolution
from scipy.sparse.linalg import eigsh

# General imports
import os
import json
import numpy as np
from datetime import datetime

from qiskit.quantum_info import SparsePauliOp

from scipy.stats.qmc import Halton

# custom module
from susy_qm import calculate_wz_hamiltonian

In [10]:
# Parameters
N = 3
a = 1.0
c = 0
potential = "linear"#'quadratic'
boundary_condition = 'dirichlet'
#boundary_condition = 'periodic'
cutoff = 2

In [11]:
shots = 1024

starttime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

if potential == 'quadratic':
    folder = 'C' + str(abs(c)) + '/' + 'N'+ str(N) + '/' + str(starttime)
else:
    folder = 'N'+ str(N) + '/' + str(starttime)

base_path = os.path.join(r"C:\Users\Johnk\Documents\Quantum Computing Code\Quantum-Computing\SUSY\Wess-Zumino\VQE\DE\Files", boundary_condition, potential, folder)
os.makedirs(base_path, exist_ok=True)

print(f"Running for {boundary_condition} boundary conditions")
print(f"Running for {potential} potential")
print(f"Running for cutoff: {cutoff}")

#calculate Hamiltonian and expected eigenvalues
H = calculate_wz_hamiltonian(cutoff, N, a, potential, boundary_condition, c)

eigenvalues = np.sort(np.linalg.eig(H)[0])[:12]
min_eigenvalue = min(eigenvalues.real)

#create qiskit Hamiltonian Pauli string
hamiltonian = SparsePauliOp.from_operator(H)
num_qubits = hamiltonian.num_qubits
print("Num qubits: ", num_qubits)

# Device
shots = shots
#dev = qml.device('default.qubit', wires=num_qubits, shots=shots)
dev = qml.device('default.qubit', wires=num_qubits, shots=shots)


sel_num_layers = 2  # Number of entangling layers
sel_params_shape = qml.StronglyEntanglingLayers.shape(n_layers=sel_num_layers, n_wires=num_qubits)

ran_num_layers = 2  # Number of entangling layers
ran_params_shape = qml.RandomLayers.shape(n_layers=ran_num_layers, n_rotations=num_qubits)


@qml.qnode(dev)
def cost_function(params):

    sel_params = params[:np.prod(sel_params_shape)]
    ran_params = params[np.prod(sel_params_shape):]
    sel_params = pnp.tensor(sel_params.reshape(sel_params_shape), requires_grad=True)
    ran_params = pnp.tensor(ran_params.reshape(ran_params_shape), requires_grad=True)

    qml.StronglyEntanglingLayers(sel_params, wires=range(num_qubits), imprimitive=qml.CZ)
    qml.RandomLayers(ran_params, wires=range(num_qubits), seed=280270)

    return qml.expval(qml.Hermitian(H, wires=range(num_qubits)))


# VQE
vqe_start = datetime.now()


num_vqe_runs = 1
max_iterations = 10000
strategy = "randtobest1bin"
popsize = 20
tol = 1e-2
abs_tol = 1e-3

#data arrays
energies = []
x_values = []
success = []
run_times = []
num_iters = []
num_evaluations = []

#Optimizer
sel_x0 = np.random.random(sel_params_shape).flatten()
ran_x0 = np.random.random(ran_params_shape).flatten()
x0 = np.concatenate((sel_x0, ran_x0))
bounds = [(0, 2 * np.pi) for _ in range(len(x0))]

# Number of dimensions and population size
num_dimensions = len(x0)
num_samples = popsize

# Generate Halton sequence
halton_sampler = Halton(d=num_dimensions, scramble=True)
halton_samples = halton_sampler.random(n=num_samples)

# Scale samples to parameter bounds (0 to 2*pi)
scaled_samples = 2 * np.pi * halton_samples

for i in range(num_vqe_runs):

    run_start = datetime.now()

    if i % 1 == 0:
        print(f"Run: {i}")

    
    # Differential Evolution optimization
    res = differential_evolution(cost_function, 
                                    bounds, 
                                    maxiter=max_iterations, 
                                    tol=tol,
                                    atol=abs_tol,
                                    strategy=strategy, 
                                    popsize=popsize,
                                    init=scaled_samples)
    

    energies.append(res.fun)
    x_values.append(res.x)
    success.append(res.success)
    num_iters.append(res.nit)
    num_evaluations.append(res.nfev)

    run_end = datetime.now()
    run_time = run_end - run_start
    run_times.append(run_time)

vqe_end = datetime.now()
vqe_time = vqe_end - vqe_start

Running for dirichlet boundary conditions
Running for linear potential
Running for cutoff: 2
Num qubits:  6
Run: 0


In [20]:
cost_function(pnp.array(x_values[0]))

array(0.28452699)

In [12]:
energies

[np.float64(0.18071565414776514)]

In [15]:
x_values

[array([1.65139339, 3.15354815, 2.38450114, 2.29795322, 0.09034982,
        3.7900682 , 2.77070644, 2.96023782, 2.9678021 , 4.59261982,
        6.04342029, 5.29319019, 4.51390138, 2.80874959, 3.84295552,
        1.21597272, 3.02680582, 5.53450261, 0.37292169, 3.20531798,
        2.22900305, 2.88715298, 3.0379538 , 3.983689  , 0.61987954,
        5.67769629, 1.10094407, 1.16347102, 0.18076868, 3.33680361,
        2.35224424, 5.9453558 , 1.17606433, 5.18458511, 3.12012092,
        1.6495082 , 3.14098932, 3.15999269, 2.51620619, 2.89422059,
        4.98504885, 4.10261593, 2.76299021, 0.93807796, 2.97910089,
        4.63569638, 5.20532848, 3.8255258 ])]

In [13]:
success

[False]

In [7]:
eigenvalues

array([1.17463681e-04+0.j, 9.07816775e-01+0.j, 9.56369363e-01+0.j,
       1.00011746e+00+0.j, 1.03278319e+00+0.j, 1.21190784e+00+0.j,
       1.22486234e+00+0.j, 1.22486234e+00+0.j, 1.31557154e+00+0.j,
       1.56874180e+00+0.j, 1.86406867e+00+0.j, 1.90781678e+00+0.j])

In [8]:
np.sort(eigenvalues.real)

array([1.17463681e-04, 9.07816775e-01, 9.56369363e-01, 1.00011746e+00,
       1.03278319e+00, 1.21190784e+00, 1.22486234e+00, 1.22486234e+00,
       1.31557154e+00, 1.56874180e+00, 1.86406867e+00, 1.90781678e+00])

In [21]:
#Save run
run = {
    'starttime': starttime,
    'potential': potential,
    'boundary_condition': boundary_condition,
    'c': c,
    'cutoff': cutoff,
    'num_sites': N,
    'exact_eigenvalues': [round(x.real,10).tolist() for x in eigenvalues],
    'ansatz': 'StronglyEntanglingLayers-1layer',
    'num_VQE': num_vqe_runs,
    'shots': shots,
    'Optimizer': {'name': 'differential_evolution',
                'bounds':'[(0, 2 * np.pi) for _ in range(np.prod(params_shape))]',
                'maxiter':max_iterations,
                'tolerance': tol,
                'strategy': strategy,
                'popsize': popsize
                },
    'results': energies,
    'params': [x.tolist() for x in x_values],
    'num_iters': num_iters,
    'num_evaluations': num_evaluations,
    'success': np.array(success, dtype=bool).tolist(),
    'run_times': [str(x) for x in run_times],
    'total_run_time': str(vqe_time)
}

# Save the variable to a JSON file
path = base_path + "{}_{}.json".format(potential, cutoff)
with open(path, 'w') as json_file:
    json.dump(run, json_file, indent=4)

In [ ]:


#base_path = r"C:\Users\Johnk\OneDrive\Desktop\PhD 2024\Quantum Computing Code\Quantum-Computing\SUSY\PennyLane\SUSY VQE\Shot Noise\Files\{}\\{}\\"
#create_vqe_plots(potential=potential, base_path=base_path, folder=folder, cut_off_list=cut_offs_list)